In [ ]:
import torch
import deepxde as dde
import numpy as np
from typing import cast, Any
import plotly.graph_objects as go
import plotly

%load_ext jupyter_black

In [ ]:
class HeatEquation1D:
    """Class modeling the heat equation in 1+1 dimensions. The PDE is:
    $$
        \\frac{\\partial y}{\\partial t} - \\kappa \\nabla^2 y = 0
    $$
    and with initial conditions $y(t = 0, x) = \\sin(n \\pi x)$ and boundary conditions $y(t, x = 0) = y(t, x = 1) = 0$.


    Args:
        kappa (float): Thermal diffusivity.
        n (int): Order of the solution.
    """

    __slots__ = ("kappa", "n")

    def __init__(self, kappa: float, n: int):
        # PDE parameters
        self.kappa = kappa
        self.n = n

    def equation(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        """Defines the ODE.

        Args:
            x (torch.Tensor): Input of the neural network.
            y (torch.Tensor): Output of the neural network.

        Returns:
            (torch.Tensor): Residual of the differential equation.

        """
        dy_dt = cast(torch.Tensor, dde.grad.jacobian(y, x, i=0, j=1))
        dy2_dx2 = cast(torch.Tensor, dde.grad.hessian(y, x, i=0, j=0))
        pde = dy_dt - self.kappa * dy2_dx2
        return pde

    def initial_value(self, x: np.ndarray) -> np.ndarray:
        """Defines the initial value for the differential equation.

        Args:
            x (np.ndarray): Input of the neural network.

        Returns:
            (np.ndarray): Value at the boundary for the initial condition.
        """
        y_init = np.sin(self.n * np.pi * x[:, 0:1])
        return y_init

    def boundary_value(self, x: torch.Tensor) -> float:
        """Defines the boundary value for the differential equation.

        Args:
            x (torch.Tensor): Input of the neural network.

        Returns:
            (float): Residual of the initial value condition for the derivative.
        """
        return 0

    def exact_solution(self, x: np.ndarray) -> np.ndarray:
        """Defines the ground truth solution for the system.

        Args:
            x (np.ndarray): Points on which to evaluate the solution.

        Returns:
            (np.ndarray): Value of the solution on the points.
        """

        y_spatial = self.initial_value(x)
        y_time = np.exp(-self.n**2 * np.pi**2 * self.kappa * x[:, 1:2])
        return y_spatial * y_time

In [ ]:
# Equation parameters

kappa = 0.4
n = 1

# Neural network parameters
input_size = [2]
output_size = [1]
layers_sizes = input_size + [16, 32, 16] + output_size
activation = "tanh"
initializer = "Glorot normal"

# Training parameters
optimizer_kw = dict(
    lr=0.001,
    metrics=["l2 relative error"],
    # loss_weights=[0.001, 1, 1],
)
n_iterations = 10_000

# Mesh parameters
n_training_inside = 2000
n_training_bdy = 80
n_training_initial = 160
n_test = 2000

# Ranges
x_begin = 0
x_end = 1
t_begin = 0
t_end = 1


def solve_problem(
    kappa: float, n: int
) -> tuple[HeatEquation1D, dde.Model, dde.model.LossHistory, dde.model.TrainState]:
    """Given the parameters, defines the model and trains the model."""
    # System
    heat_eq = HeatEquation1D(kappa=kappa, n=n)

    # Geometry description
    geom_space = dde.geometry.Interval(x_begin, x_end)
    geom_time = dde.geometry.TimeDomain(t_begin, t_end)
    geom_full = dde.geometry.GeometryXTime(geom_space, geom_time)

    # Initial value conditions
    bc = dde.icbc.DirichletBC(
        geom_full, heat_eq.boundary_value, lambda _, on_boundary: on_boundary
    )
    ic = dde.icbc.IC(
        geom_full,
        heat_eq.initial_value,
        lambda _, on_initial: on_initial,
    )

    # Load data
    data = dde.data.TimePDE(
        geom_full,
        heat_eq.equation,
        [bc, ic],
        num_domain=n_training_inside,
        num_boundary=n_training_bdy,
        num_initial=n_training_initial,
        solution=heat_eq.exact_solution,
        num_test=n_test,
    )

    neural_net = dde.nn.FNN(layers_sizes, activation, initializer)
    # neural_net.apply_output_transform(lambda x, y: abs(y))
    model = dde.Model(data, neural_net)
    model.compile("adam", **optimizer_kw)
    # model.train(iterations=n_iterations)
    # model.compile("L-BFGS")
    losshistory, train_state = model.train(iterations=n_iterations)

    return heat_eq, model, losshistory, train_state

In [ ]:
def get_predictions(
    heat_eq: HeatEquation1D, model: dde.Model, n_dim: int
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    x = np.linspace(x_begin, x_end, n_dim)
    t = np.linspace(t_begin, t_end, n_dim)
    X, T = np.meshgrid(x, t)
    grid = np.stack([X.flatten(), T.flatten()], axis=1)

    y_true = heat_eq.exact_solution(grid)
    y_true = y_true.reshape((n_dim, n_dim))
    y_predict = model.predict(grid)
    y_predict = y_predict.reshape((n_dim, n_dim))
    return (x, t), y_predict, y_true

In [ ]:
def mae(y_true: np.ndarray, y_predict: np.ndarray) -> float:
    return np.mean(np.abs(y_true - y_predict))


def plot_result(
    coords: np.ndarray, y_predict: np.ndarray, y_true: np.ndarray, n: int, kappa: float
) -> go.Figure:
    width = 600
    height = 600
    scale = 3

    fig = go.Figure(
        data=go.Heatmap(
            z=y_predict.T,
            x=coords[1],
            y=coords[0],
            colorscale="RdBu_r",
            zmin=-1,
            zmax=1,
            colorbar=dict(
                title=dict(text="Temperature difference", side="bottom"),
                orientation="h",
                x=0.5,
                y=-0.2,
                xanchor="center",
                yanchor="top",
            ),
        )
    )
    error = mae(y_true.flatten(), y_predict.flatten())
    fig.update_layout(
        autosize=False,
        width=width,
        height=height,
        title=dict(
            text=f"Case n={n}, diffusivity = {kappa:.1E}, MAE: {error:.2E}",
            y=0.91,
            x=0.5,
            xanchor="center",
            yanchor="bottom",
        ),
        xaxis=dict(title=dict(text="t")),
        yaxis=dict(title=dict(text="x")),
        font=dict(size=14),
        margin=dict(l=40, r=40, t=60, b=60),
    )
    fig.write_image(
        f"../figs/heat_equation_n={n}.png",
        width=width,
        height=height,
        scale=scale,
    )
    return fig

In [ ]:
n = 1
kappa = 0.1
# Training parameters
optimizer_kw = dict(
    lr=0.001,
    metrics=["l2 relative error"],
    # loss_weights=[0.001, 2, 0.5],
)
heat_eq, model, loss_history, train_state = solve_problem(kappa=kappa, n=n)
dde.saveplot(loss_history, train_state, issave=False, isplot=True)
# plot_result(train_state, name_case="underdamped")

In [ ]:
coords, y_predict, y_true = get_predictions(heat_eq, model, 1000)
plot_result(coords, y_predict, y_true, n=1, kappa=0.1)

In [ ]:
n = 4
kappa = 0.025
# Training parameters
optimizer_kw = dict(
    lr=0.001,
    metrics=["l2 relative error"],
    # loss_weights=[0.001, 2, 0.5],
)
heat_eq_2, model_2, loss_history_2, train_state_2 = solve_problem(kappa=kappa, n=n, kappa=kappa)

In [ ]:
dde.saveplot(loss_history_2, train_state_2, issave=False, isplot=True)
# plot_result(train_state, name_case="underdamped")

coords, y_predict, y_true = get_predictions(heat_eq_2, model_2, 1000)
plot_result(coords, y_predict, y_true, n=4, kappa=0.025)